In [5]:
import os
import yfinance as yf
import pandas as pd
from langchain_core.tools import tool

@tool
def get_trade_signals(ticker: str):
    """
    Calculates Support (Entry), Resistance (Target), and Stop Loss.
    """
    try:
        stock = yf.Ticker(ticker)
        # Fetch 1 year of data to calculate valid 60-day support
        hist = stock.history(period="1y")
        
        if hist.empty: return "Error: No Data found."

        # --- INDICATORS ---
        # 1. Trend (SMA 50)
        hist['SMA_50'] = hist['Close'].rolling(window=50).mean()
        
        # 2. Support & Resistance (60-day Lookback)
        hist['Support'] = hist['Low'].rolling(window=60).min()
        hist['Resistance'] = hist['High'].rolling(window=60).max()
        
        # 3. Bollinger Bands (Volatility)
        hist['SMA_20'] = hist['Close'].rolling(window=20).mean()
        hist['STD_20'] = hist['Close'].rolling(window=20).std()
        hist['BB_Lower'] = hist['SMA_20'] - (hist['STD_20'] * 2)
        
        # Current Data Points
        curr = hist.iloc[-1]
        price = curr['Close']
        
        # --- TRADING LOGIC ---
        # Rule: Stop Loss is 3% below the Major Support Level
        stop_loss = curr['Support'] * 0.97
        
        # Rule: Entry Logic
        signal_type = "WAIT / NEUTRAL"
        if price <= curr['BB_Lower']:
            signal_type = "BUY (Oversold - Bollinger Band)"
        elif price <= (curr['Support'] * 1.02): # Within 2% of Support
            signal_type = "BUY (Near Support Zone)"
        elif price >= (curr['Resistance'] * 0.98):
            signal_type = "SELL (Near Resistance)"
            
        return (
            f"--- TECHNICAL DATA FOR {ticker} ---\n"
            f"Price: {price:.2f}\n"
            f"Signal: {signal_type}\n"
            f"Support (Entry): {curr['Support']:.2f}\n"
            f"Resistance (Target): {curr['Resistance']:.2f}\n"
            f"Stop Loss (3% Risk): {stop_loss:.2f}\n"
        )
    except Exception as e:
        return f"Error: {e}"

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

def ingest_knowledge_base():
    """Reads PDF and creates a searchable database."""
    if os.path.exists("./chroma_db"):
        print("✅ Database already exists. Skipping ingestion.")
        return

    if not os.path.exists("./data"):
        os.makedirs("data")
        print("⚠️ created 'data' folder. Please put a PDF there and run again.")
        return

    try:
        print("⚡ Ingesting PDFs...")
        # Load all PDFs in the folder
        loaders = [PyPDFLoader(os.path.join("./data", f)) for f in os.listdir("./data") if f.endswith(".pdf")]
        docs = []
        for loader in loaders:
            docs.extend(loader.load())
            
        # Split text into chunks
        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        splits = splitter.split_documents(docs)
        
        # Embed and Save
        embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        Chroma.from_documents(splits, embedding, persist_directory="./chroma_db")
        print("✅ Database Created Successfully.")
    except Exception as e:
        print(f"Error during ingestion: {e}")

ingest_knowledge_base()
print("✅ Phase 2 Complete: Tools & DB Ready.")

⚡ Ingesting PDFs...
✅ Database Created Successfully.
✅ Phase 2 Complete: Tools & DB Ready.
